In [1]:
from netgen.occ import *
from ngsolve import Mesh

import sys
sys.path.append("../build/rigid_body")
from rigid_body import *
import rigid_body.bla as bla

import pythreejs as p3

from IPython.display import display
import ipywidgets as widgets

## Generate Geometry

In [2]:
# set up OCC CAD model
box = Box(Pnt(-0.5,-0.5,-0.5), Pnt(0.5,0.5,0.5))
geo = OCCGeometry(box)
mesh = Mesh(geo.GenerateMesh(maxh=0.5))
# important: move the center of mass into the origin
box = box.Move((-box.center[0], -box.center[1], -box.center[2]))

In [3]:
# from ngsolve.webgui import Draw
# Draw(mesh)

In [4]:
def body_from_solid(obj):
    "extracts the mass matrix of the TopAbs_ShapeEnum.SOLID obj, using the figures computed by netgen"
    
    # copy the inertia matrix from netgen
    inertia_matrix = bla.Matrix(3,3)
    for i in range(3):
        for j in range(3):
            inertia_matrix[i, j] = obj.inertia[i, j]

    # copy the center of mass from netgen
    center_of_mass = bla.Vector(3)
    for i in range(3): center_of_mass[i] = obj.center[i]

    # rearrange it in C++ to make the mass matrix (the elegant way, using MatrixView)
    body = RigidBody()
    for i in range(3) : body.center[i] = obj.center[i]
    body.mass = obj.mass
    body.inertia = inertia_matrix
    body.recalcMassMatrix()
    
    return body


In [5]:
def extract_vertices(mesh: Mesh):
    "extracts a p3js compatible vertex list from a netgen.occ Mesh"
    combined_vertices = [] # flat array of vertices of all faces
    all_vertices = [vert.point for vert in mesh.vertices] # all available vertices

    # throw all vertices of all faces into combined_vertices
    for face in mesh.faces:
        for vertex in face.vertices:
            point = all_vertices[vertex.nr]
            combined_vertices.append(point)
            
    return combined_vertices

## Setup System

In [6]:
rbs = RBSystem()
rbs.gravity = (0,1,0)
# set up physics simulation object
r1 = body_from_solid(box)

p1 = Transformation()
p1.setTranslation(0,0,0)
p1.setRotation(0,0,1)
p1.setRotation(1,1,1)
p1.setRotation(2,2,1)
r1.q=p1

t1 = Transformation()
#t1.setTranslation(0.1,0.0,0.0)
#t1.setRotation(0,2,0.001)
#t1.setRotation(2,0,-0.001)
r1.dq=t1


r2 = body_from_solid(box)

p2 = Transformation()
p2.setTranslation(2,2,2)
p2.setRotation(0,0,1)
p2.setRotation(1,1,1)
p2.setRotation(2,2,1)
r2.q=p2

t2 = Transformation()
t2.setTranslation(0.05,0.0,0)
t2.setRotation(0,2,2)
t2.setRotation(2,0,-2)
r2.dq=t2
#Add body to system
c1 = rbs.addBody(r1)
c1.pos=(0.5,0.5,0.5)
c2 = rbs.addBody(r2)
c2.pos=(-0.5,-0.5,-0.5)
b = Beam(c1,c2,1.7320508075688772);
rbs.addBeam(b)
fix = rbs.addFix()
#fix.pos = (2,4,2)
#c2.pos = (0,0.5,0)
#s = Spring(fix,c2,2,0.8)
#rbs.addSpring(s)


r3 = body_from_solid(box)

pos3 = Transformation()
pos3.setTranslation(2.5,3,3.5)
pos3.setRotation(0,0,1)
pos3.setRotation(1,1,1)
pos3.setRotation(2,2,1)
r3.q=pos3

t3 = Transformation()
t3.setTranslation(1,0.0,0.0)
#t1.setRotation(0,2,0.001)
#t1.setRotation(2,0,-0.001)
r3.dq=t3

#c3 = rbs.addBody(r3)
# save a restore point for the reset button
rbs.saveState()

#array of meshed for the rigidbodies
meshes = [mesh,mesh]

## Setup Graphics

In [7]:
def appendConnector(c,connectors):
    p = rbs.connectorPos(c);
    if(c.type == 0):
        color = 'green'
    else :
        color = 'black'
    connectors.append(
        p3.Mesh(p3.SphereBufferGeometry(0.2, 16, 16),
             p3.MeshStandardMaterial(color=color),
             position=(p[0], p[1], p[2])))

def initConnectors(l):
    connectors = []
    for s in l:
        cA = s.connectorA
        cB = s.connectorB
        appendConnector(cA,connectors);
        appendConnector(cB,connectors);
    return connectors
    

def updateConnectors(l, connectors):
    for i in range(len(l)):
        cA = l[i].connectorA
        cB = l[i].connectorB
        pA = rbs.connectorPos(cA);
        pB = rbs.connectorPos(cB);
        connectors[2*i].position = (pA[0], pA[1], pA[2])
        connectors[2*i+1].position = (pB[0], pB[1], pB[2])

def positionsOf(l):
    res = []
    for s in l:
        cA = s.connectorA
        cB = s.connectorB
        pA = rbs.connectorPos(cA);
        pB = rbs.connectorPos(cB);
        res.append ([ [pA[0], pA[1], pA[2]], [pB[0], pB[1], pB[2]] ])
    return res


connectorsSprings = initConnectors(rbs.springs())
springpos = positionsOf(rbs.springs())

if springpos:
    springgeo = p3.LineSegmentsGeometry(positions=springpos)
    m2 = p3.LineMaterial(linewidth=3, color='cyan')
    springs = p3.LineSegments2(springgeo, m2)

connectorsBeams = initConnectors(rbs.beams())
beampos = positionsOf(rbs.beams())

if beampos:
    beamgeo = p3.LineSegmentsGeometry(positions=beampos)
    m2 = p3.LineMaterial(linewidth=4, color='blue')
    beams = p3.LineSegments2(beamgeo, m2)    


In [8]:
view_width = 600
view_height = 400
buffergeos = []
p3meshes = []
# set up pythreejs 3d object
for m in meshes:
    original_vertices = extract_vertices(m)
    buffergeom = p3.BufferGeometry(attributes = {"position" : p3.BufferAttribute(original_vertices, normalized=False)})
    buffergeom.exec_three_obj_method("computeVertexNormals") # gives normals for shading
    material = p3.MeshPhongMaterial(color='#ff3333', shininess=150, morphTargets=True, side="DoubleSide")
    p3mesh = p3.Mesh(buffergeom, material, position=(0,0,0))
    buffergeos.append(buffergeom)
    p3meshes.append(p3mesh)

# extra scene contents
camera = p3.PerspectiveCamera( position=[10, 6, 10], aspect=view_width/view_height)
key_light = p3.DirectionalLight(position=[0, 10, 10])
ambient_light = p3.AmbientLight()
grid = p3.GridHelper(500, 500//5, "#2F4F4F", "#2F4F4F")

# set up scene
scene = p3.Scene(children=[camera, key_light, ambient_light, grid,*p3meshes, *connectorsSprings,*connectorsBeams]+ ([] if not beampos else [beams]) + ([] if not springpos else [springs]))


axesHelper = p3.AxesHelper( 5 );
scene.add( axesHelper );
controller = p3.OrbitControls(controlling=camera)
renderer = p3.Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height, antialias=True) # if performance is bad, try removing antalias=True

In [9]:
print(r1.dq)

 Translation: 	0.000000 ,0.000000, , 0.000000
 Rotation: 	0.000000 ,0.000000, , 0.000000
 		0.000000 ,0.000000, , 0.000000
		0.000000 ,0.000000, , 0.000000



In [10]:
# play/reset button
play = widgets.Play(
    value=10,
    min=0,
    max=10000,
    step=1,
    interval=100/30,
    description="Press play",
    disabled=False
)

In [11]:
springpos

[]

In [12]:
for m in p3meshes:
    m.matrixAutoUpdate = False # make mesh movable

def refresh():
    for i in range(0,len(meshes)):
        p3meshes[i].matrix=rbs.bodies()[i].q.asTuple()
    updateConnectors(rbs.springs(),connectorsSprings)
    springpos = positionsOf(rbs.springs())
    if springpos:
        springs.geometry = p3.LineSegmentsGeometry(positions=springpos)
    
    updateConnectors(rbs.beams(),connectorsBeams)
    beampos = positionsOf(rbs.beams())
    if beampos:
        beams.geometry = p3.LineSegmentsGeometry(positions=beampos)

def update():
    "update function, gets called every timestep; quasi main loop"
    rbs.simulate(0.05,1)
    refresh()

def observer(state):
    "event handler for clickable buttons"
    # if there is a change in time
    if state["name"] == "value":
        # it might be a reset
        if str(state["new"]) == "0":
            rbs.reset()
            refresh()
        # or it might be a progress in time
        else:
            update()
    # repeat is used as an alias to reset
    elif state["name"] == "repeat":
        rbs.reset()
        refresh()

play.observe(observer)


## Renderer

In [13]:
display(widgets.HBox([play, widgets.HTML("<b>click-and-drag to rotate, scroll to zoom, right-click-and-drag to move<b>")]))
display(renderer)
refresh()

Renderer(camera=PerspectiveCamera(aspect=1.5, position=(10.0, 6.0, 10.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0…